In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from huggingface_hub import login
import os

hf_token = os.getenv("hf_token")

login(token=hf_token)

In [3]:
! pip install -q kaggle

In [4]:
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c glioma-mcd-2025

 69% 262M/382M [00:00<00:00, 1.37GB/s]
100% 382M/382M [00:00<00:00, 1.36GB/s]


In [6]:
!unzip glioma-mcd-2025.zip -d /content

Streaming output truncated to the last 5000 lines.
  inflating: /content/Data_122824/Glioma_MDC_2025_training/training0580.jpg  
  inflating: /content/Data_122824/Glioma_MDC_2025_training/training0580.json  
  inflating: /content/Data_122824/Glioma_MDC_2025_training/training0581.jpg  
  inflating: /content/Data_122824/Glioma_MDC_2025_training/training0581.json  
  inflating: /content/Data_122824/Glioma_MDC_2025_training/training0582.jpg  
  inflating: /content/Data_122824/Glioma_MDC_2025_training/training0582.json  
  inflating: /content/Data_122824/Glioma_MDC_2025_training/training0583.jpg  
  inflating: /content/Data_122824/Glioma_MDC_2025_training/training0583.json  
  inflating: /content/Data_122824/Glioma_MDC_2025_training/training0584.jpg  
  inflating: /content/Data_122824/Glioma_MDC_2025_training/training0584.json  
  inflating: /content/Data_122824/Glioma_MDC_2025_training/training0585.jpg  
  inflating: /content/Data_122824/Glioma_MDC_2025_training/training0585.json  
  infla

In [7]:
!kaggle competitions download -c glioma-mdc-2025-one-shot-testing

  0% 0.00/19.6M [00:00<?, ?B/s]
100% 19.6M/19.6M [00:00<00:00, 1.18GB/s]


In [8]:
!unzip glioma-mdc-2025-one-shot-testing.zip -d /content/ost

Archive:  glioma-mdc-2025-one-shot-testing.zip
  inflating: /content/ost/Oneshot_testing/Oneshot_testing/0001.jpg  
  inflating: /content/ost/Oneshot_testing/Oneshot_testing/0001.json  
  inflating: /content/ost/Oneshot_testing/Oneshot_testing/0002.jpg  
  inflating: /content/ost/Oneshot_testing/Oneshot_testing/0002.json  
  inflating: /content/ost/Oneshot_testing/Oneshot_testing/0003.jpg  
  inflating: /content/ost/Oneshot_testing/Oneshot_testing/0003.json  
  inflating: /content/ost/Oneshot_testing/Oneshot_testing/0004.jpg  
  inflating: /content/ost/Oneshot_testing/Oneshot_testing/0004.json  
  inflating: /content/ost/Oneshot_testing/Oneshot_testing/0005.jpg  
  inflating: /content/ost/Oneshot_testing/Oneshot_testing/0005.json  
  inflating: /content/ost/Oneshot_testing/Oneshot_testing/0006.jpg  
  inflating: /content/ost/Oneshot_testing/Oneshot_testing/0006.json  
  inflating: /content/ost/Oneshot_testing/Oneshot_testing/0007.jpg  
  inflating: /content/ost/Oneshot_testing/Oneshot_

In [9]:
from PIL import Image as PILImage
from huggingface_hub import hf_hub_download, from_pretrained_keras
import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
import cv2
from PIL import Image
import glob


In [10]:
sub = pd.read_csv('/content/ost/submission_template.csv')
sub_dict = {}
for index, row in sub.iterrows():
    sub_dict[str(row['Image ID']).zfill(4)+'_'+row['Label ID']] = index

In [11]:
train_images = sorted(glob.glob('/content/Data_122824/Glioma_MDC_2025_training/*.jpg'))
train_json = sorted(glob.glob('/content/Data_122824/Glioma_MDC_2025_training/*.json'))
test_images = sorted(glob.glob('/content/ost/Oneshot_testingV2/Test-2.1/*.jpg'))
test_json = sorted(glob.glob('/content/ost/Oneshot_testingV2/Test-2.1/*.json'))
len(train_images), len(train_json), len(test_images), len(test_json)

(1439, 1439, 95, 95)

In [12]:
labels_dict = {'Non-mitosis':0, 'Mitosis':1}

In [13]:
# Load the model directly from Hugging Face Hub
loaded_model = from_pretrained_keras("google/path-foundation")

# Call inference
infer = loaded_model.signatures["serving_default"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

variables.index:   0%|          | 0.00/3.64k [00:00<?, ?B/s]

saved_model.pb:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/12.9k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

variables.data-00000-of-00001:   0%|          | 0.00/110M [00:00<?, ?B/s]

Test.png:   0%|          | 0.00/208k [00:00<?, ?B/s]

fingerprint.pb:   0%|          | 0.00/58.0 [00:00<?, ?B/s]

keras_metadata.pb:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

In [14]:
# Function to rescale coordinates if needed
def rescale_coordinates(points, original_size, target_size):
    """
    Rescale the points from the original size to the target size.
    """
    if original_size != target_size:
        scale_factor = target_size / original_size
        return [(int(x * scale_factor), int(y * scale_factor)) for x, y in points]
    else:
        return [(int(x), int(y)) for x, y in points]

In [15]:
All_labels = []
itr = 0
train_embeddings = np.zeros((1616,384))
for i in range(len(train_images)):
    image_path = train_images[i]
    json_path = train_json[i]
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

    with open(json_path, 'r') as f:
        data = json.load(f)

    # Determine if rescaling is needed
    json_image_height = data['imageHeight']
    json_image_width = data['imageWidth']
    actual_image_height, actual_image_width, _ = image.shape

    # Rescale coordinates if the JSON dimensions differ from the actual image dimensions
    shapes = data['shapes']
    labels = [labels_dict[shape['label']] for shape in shapes]
    rois = [
        rescale_coordinates(shape['points'], json_image_height, actual_image_height)
        for shape in shapes
    ]
    All_labels = All_labels + labels
    for points in rois:
      points = np.array(points, dtype=np.int32)
      x_min, y_min = np.min(points, axis=0)
      x_max, y_max = np.max(points, axis=0)
      cropped = image[y_min:y_max, x_min:x_max]
      cropped = Image.fromarray(cropped)
      cropped = cropped.resize((224, 224))
      tensor = tf.cast(tf.expand_dims(np.array(cropped), axis=0), tf.float32) / 255.0
      embeddings = infer(tf.constant(tensor))
      train_embeddings[itr] = embeddings['output_0'].numpy().flatten()
      itr += 1
len(All_labels), itr

(1616, 1616)

In [16]:
itr = 0
test_indices = []
test_embeddings = np.zeros((1444,384))
for i in range(len(test_images)):
    image_path = test_images[i]
    json_path = test_json[i]
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

    with open(json_path, 'r') as f:
        data = json.load(f)

    # Determine if rescaling is needed
    json_image_height = data['imageHeight']
    json_image_width = data['imageWidth']
    actual_image_height, actual_image_width, _ = image.shape

    # Rescale coordinates if the JSON dimensions differ from the actual image dimensions
    shapes = data['shapes']
    ImageID = image_path.split('/')[-1].split('.')[0]
    labels = [sub_dict[ImageID+'_'+shape['label']] for shape in shapes]
    rois = [
        rescale_coordinates(shape['points'], json_image_height, actual_image_height)
        for shape in shapes
    ]
    test_indices = test_indices + labels
    for points in rois:
      points = np.array(points, dtype=np.int32)
      x_min, y_min = np.min(points, axis=0)
      x_max, y_max = np.max(points, axis=0)
      cropped = image[y_min:y_max, x_min:x_max]
      cropped = Image.fromarray(cropped)
      cropped = cropped.resize((224, 224))
      tensor = tf.cast(tf.expand_dims(np.array(cropped), axis=0), tf.float32) / 255.0
      embeddings = infer(tf.constant(tensor))
      test_embeddings[itr] = embeddings['output_0'].numpy().flatten()
      itr += 1
len(test_indices), itr

(137, 137)

In [17]:
All_labels2 = np.array(All_labels)

In [18]:
!pip install tabpfn
!git clone https://github.com/priorlabs/tabpfn-extensions.git
!pip install -e tabpfn-extensions
%cd tabpfn-extensions/src
from tabpfn import TabPFNClassifier
from tabpfn_extensions.post_hoc_ensembles.sklearn_interface import AutoTabPFNClassifier
%cd ../..

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 130.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninst

Cloning into 'tabpfn-extensions'...
remote: Enumerating objects: 2151, done.
remote: Counting objects: 100% (355/355), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 2151 (delta 246), reused 228 (delta 204), pack-reused 1796 (from 2)
Receiving objects: 100% (2151/2151), 806.69 KiB | 2.55 MiB/s, done.
Resolving deltas: 100% (1306/1306), done.
Obtaining file:///content/tabpfn-extensions
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for tabpfn-extensions (pyproject.toml) ... done
  Created wheel for tabpfn-extensions: filename=tabpfn_extensions-0.1.0-0.editable-py3-none-any.whl size=11662 sha256=3b10e7af970e335952f6246c118c31e2435445d1587136c68d4b16d0da50b901
  Stored in directory: /tmp/pip-ephem-wheel-cache-2_alaaiq/wheels/8f/b7/2c/ec18b502697068b39e1503426a50b729bdac046bed4810759d
Suc

In [19]:
model = AutoTabPFNClassifier(device="cuda",max_time=600)
model.fit(train_embeddings, All_labels2)
test_probs = model.predict_proba(test_embeddings)
test_probs

/usr/local/lib/python3.11/dist-packages/tabpfn/base.py:89: UserWarning: Downloading model to /root/.cache/tabpfn/tabpfn-v2-classifier.ckpt.
  model, _, config_ = load_model_criterion_config(


tabpfn-v2-classifier.ckpt:   0%|          | 0.00/29.0M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/tabpfn/base.py:89: UserWarning: Downloading model to /content/tabpfn-extensions/src/tabpfn_extensions/hpo/hpo_models/tabpfn-v2-classifier-od3j1g5m.ckpt.
  model, _, config_ = load_model_criterion_config(


tabpfn-v2-classifier-od3j1g5m.ckpt:   0%|          | 0.00/29.0M [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/tabpfn/base.py:89: UserWarning: Downloading model to /content/tabpfn-extensions/src/tabpfn_extensions/hpo/hpo_models/tabpfn-v2-classifier-gn2p4bpt.ckpt.
  model, _, config_ = load_model_criterion_config(


tabpfn-v2-classifier-gn2p4bpt.ckpt:   0%|          | 0.00/12.9M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

INFO:tabpfn_extensions.post_hoc_ensembles.greedy_weighted_ensemble:Order of selections: [np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(5), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(5), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0)]
INFO:tabpfn_extensions.post_hoc_ensembles.greedy_weighted_ensemble:Val loss over iterations: [np.float64(-0.9960884353741496), np.float64(-0.9960884353741496), np.float64(-0.9960884353741496), np.float64(-0.9960884353741496), np.float64(-0.9960884353741496), np.float64(-0.9960884353741496), np.float64(-0.9960884353741496), np.float64(-0.9960884353741496), np.float64(-0.9960884353741496), np.float64(-0.9960884353741496), np.float64(-0.9960884353741496), np.float64(-0.9960884353741496), np.float64(-0.9960884353741496), np.float64(-0.9960884353741496), np.float64(-0.9960884353741496

array([[9.99833226e-01, 1.66769620e-04],
       [9.96619165e-01, 3.38085787e-03],
       [9.99777257e-01, 2.22799063e-04],
       ...,
       [7.18347490e-01, 2.81652510e-01],
       [7.18347490e-01, 2.81652510e-01],
       [7.18347490e-01, 2.81652510e-01]])

In [20]:
predictions = test_probs.argmax(axis=1)

In [21]:
for i in range(len(test_indices)):
  sub.at[test_indices[i], 'Prediction'] = predictions[i]

In [22]:
sub.to_csv('submission.csv', index=False)

In [23]:
!kaggle competitions submit -c glioma-mdc-2025-one-shot-testing -f submission.csv  -m "google path - 1fold - autotabpfn 600"

100% 2.32k/2.32k [00:00<00:00, 4.11kB/s]
Successfully submitted to Glioma-MDC 2025 (One-shot testing)